In [ ]:
# %matplotlib notebook

from source.DCTspectral import DCTspectral
from source.FFTspectral import FFTspectral

import numpy as np
np.random.seed(0)
np.set_printoptions(threshold=12)

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
# matplotlib.use("Agg")
plt.rcParams['figure.figsize'] = [6, 6]
# plt.rcParams["animation.embed_limit"] = 50

# Initializing the spectral model

In [ ]:
Lx=Ly=1
Nx=Ny=512
eps=0.005
# eps = 4*0.0026*200/(2*np.sqrt(2)*np.arctanh(0.9)) * factor

system = FFTspectral(Lx, Ly, Nx, Ny, eps, 'random')
system.u = np.random.uniform(-1, 1, [Nx, Ny])
# system.u = np.random.uniform(-.1, .1, [Nx, Ny])+.21
# system.u = np.tanh(np.random.normal(size=[Nx, Ny]))
# system.u = np.random.choice([-1,1],size=Nx*Ny).reshape([Nx,Ny])
# system.step = system.step_method2

I = system.u.sum()

In [ ]:
img_fig = plt.figure(1)
cnt_fig = plt.figure(2)
img_axs = img_fig.add_subplot(111)
cnt_axs = cnt_fig.add_subplot(111)

# img_axs.set_title("Image")
img_txt = img_axs.text(0.02, -0.09, f"Elapsed time: {system.t}", verticalalignment='top')
img_axs.set_xlabel("X")
img_axs.set_ylabel("Y")

# cnt_axs.set_title("Contour")
cnt_txt = cnt_axs.text(0.02, -0.09, f"Elapsed time: {system.t}", verticalalignment='top')
cnt_axs.set_xlabel("X")
cnt_axs.set_ylabel("Y")

plt.close("all")

In [ ]:
cmap='gray'

img = img_axs.imshow(system.u, extent=(0, Lx, 0, Ly), cmap=cmap, origin='lower', animated=True)
def img_anim(i):
    system.evolve(reps, dt, Nt)
    img_txt.set_text(f"Elapsed time: {system.t:.0f}")
    img.set_data(system.u)
    return

n=2
levels=np.arange(-n,n)
levels=np.array([np.tanh(c) for c in levels])
# levels=[-0.9, -.45, -0.35, 0, 0.35, .45, 0.9]
# levels=[-0.9, -0.35, 0, 0.35, 0.9]

cnt_axs.contour(system.y, system.x, system.u, levels=levels, cmap='gray')
"""Change between contour and contourf here and inside cnt_anim!"""
"""countourf is usually similar to imshow, depending on the contours."""

def cnt_anim(i):
    system.evolve(reps, dt, Nt)
    cnt_axs.clear()
    cnt_axs.contour(system.y, system.x, system.u, levels=levels, cmap='gray')
    cnt_txt.set_text(f"Elapsed t: {system.t:.5f}")
    return

# Animation

In [ ]:
reps, dt, Nt, frames = 1, 1e-3, 1, 50

# anim = animation.FuncAnimation(fig=cnt_fig, func=cnt_anim, frames=5)
anim = animation.FuncAnimation(fig=img_fig, func=img_anim, frames=frames)

HTML(anim.to_jshtml())

# Monitoring $\phi$

In [ ]:
"""Press Ctrl+Return on this cell to monitor the concentrations"""
temp = f"Total: {system.u.size}", f"Above viable: {system.u[system.u>1].size}", f"Below viable: {system.u[system.u<-1].size}"
print(temp)

print(f"Mass variation, should be 0: {(system.u.sum()-I)/(Nx*Ny)}")

"""Normally the error is around the magnitude of the 64 bit float precision.
In base 2 it normally is of the order of 2^-53"""
print(f"Error in log base 2: {np.log(np.abs((system.u.sum()-I)/(Nx*Ny)))/np.log(2)}")

In [ ]:
histogram = plt.hist(system.u.flatten(), bins=int(Nx/8), density=True)

# Analysis of the Neumann boundary conditions

In order to have
$$\int_{\partial \Omega} \frac{\partial \phi}{\partial n}ds = 0, $$
we choose to set the differential of $\phi$ in the normal direction on the borders to be zero as our boundary condition.
Numerically this must be at least a close approximation.
Namely that

$$ \frac{\partial \phi}{\partial n} \approx 0 $$ 



where $\frac{\partial \phi}{\partial n}$ is the partial derivative of the order parameter in the normal direction to the boundary.

This approximation in closer to truth in our DCT spectral method rather than the FFT one.

In [ ]:
del_x = np.diff(system.u, axis=0)
del_y = np.diff(system.u)
print("L:", del_x[0,:])
print("R:", del_x[-1,:])
print("D:", del_y[:,0])
print("U:", del_y[:,-1], '\n')
# system.u[0:2,:], system.u[-3:-1,:], system.u[:, 0:2], system.u[:, -3:-1]

# Choose ord=1 or ord=2
measure = lambda vec: np.linalg.norm(vec, ord=2)
print("L:", measure(del_x[0,:]), "R:", measure(del_x[-1,:]), "D:", measure(del_y[:,0]), "U:", measure(del_y[:,-1]))

"""The actual sum, or integral, over the boundary of del phi del n:"""
measure = lambda vec: np.sum(vec)
print("L:", measure(del_x[0,:]), "R:", measure(del_x[-1,:]), "D:", measure(del_y[:,0]), "U:", measure(del_y[:,-1]))

In [ ]:
plt.imshow(del_x, extent=(0, Lx, 0, Ly), cmap=cmap, origin='lower')
plt.text(0.02, -0.09, f"Elapsed time: {system.t:.5f}")
plt.show()

In [ ]:
plt.imshow(del_y, extent=(0, Lx, 0, Ly), cmap=cmap, origin='lower')
plt.text(0.02, -0.09, f"Elapsed time: {system.t:.5f}")
plt.show()

In [ ]:
del_x = np.diff(system.u, axis=0)
del_y = np.diff(system.u)
print("L:", del_x[0,:])
print("R:", del_x[-1,:])
print("D:", del_y[:,0])
print("U:", del_y[:,-1], '\n')
# system.u[0:2,:], system.u[-3:-1,:], system.u[:, 0:2], system.u[:, -3:-1]

# Choose ord=1 or ord=2
measure = lambda vec: np.linalg.norm(vec, ord=2)
print("L:", measure(del_x[0,:]), "R:", measure(del_x[-1,:]), "D:", measure(del_y[:,0]), "U:", measure(del_y[:,-1]))

"""The actual sum, or integral, over the boundary of del phi del n:"""
measure = lambda vec: np.sum(vec)
print("L:", measure(del_x[0,:]), "R:", measure(del_x[-1,:]), "D:", measure(del_y[:,0]), "U:", measure(del_y[:,-1]))

# Plotting $|k|^2$ in the wavelength domain.
Normally
$k_{pq} = 2 \pi p/N_p \vec{p} + 2 \pi q/N_q \vec{q}$
for an interval of integers,
which is what we do in our DCT method,
but for FFT we include negative values for $p$ and $q$ in $k$,
which are in fact redundant (conjugates) since the space domain is real-valued.

$p$ and $q$ are the axes, they represent the coefficients of the matricial representation of $\hat{\phi}$.

$\hat{\phi}_{pq}$ has therefore wavelength $k_{pq}$.

In [ ]:
plot = 0
plt.xlabel("x interval")

if plot==1:
    plt.contourf(system.y, system.x, system.k2**2, cmap=plt.cm.gray)
elif plot==2:
    plt.contour(system.y, system.x, system.k2**2, cmap=plt.cm.gray)
else:
    plt.imshow(system.k2**2, extent=(0,1,0,1), cmap=plt.cm.gray, origin='lower')
plt.show()

# Animation from data list

In [ ]:
"""Animation from list of data, each representing a frame"""
"""Data here is the second order temporal derivative of self.u"""

cmap='gray'

temp = system.u
past_u = system.u
data = [(system.u - 2*past_u + temp)/dt**2]
times = []

img = img_axs.imshow(data, extent=(0, Lx, 0, Ly), cmap=cmap, origin='lower', animated=True)

for i in range(0, frames):
    temp = past_u
    past_u = system.u
    system.evolve(reps, dt, Nt)

    data += [(system.u - 2*past_u + temp)/dt**2]
    times += [system.t]

def img_anim(i):
    img.set_data(data[i])
    img_txt.set_text(f"Elapsed time: {times[i]}")